<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/all_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix,accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,GridSearchCV
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

In [0]:
def evaluate(y_test,y_pred):
  index=["E","N","NE","NW","S","SE","SW","W"]
  print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=index, columns=index))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_test, y_pred))," Model Accuracy=32.17%")
  print("****************")
  results= precision_recall_fscore_support(y_test, y_pred, average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print("Average precision =","{:.2%}".format(df["Precision"].mean())," Model precision=21.03%")
  print("Average recall =","{:.2%}".format(df["Recall"].mean())," Model recall=21.71%")
  print(df)
  

In [3]:

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
dr=[drive_4km,drive_1km]

In [0]:
y_data=pd.read_excel(dr[1]+"y_data.xlsx")
x_data=pd.read_csv(dr[1]+"x_data",usecols=range(1,10))

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="date")

**Display**

In [0]:
pd.concat([x_data,y_data],axis=1).set_index('date')

**Classifier SVM**

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test,y_pred)


    E  N  NE  NW  S  SE  SW  W
E   0  0  12   0  0   0   0  0
N   0  0  12   0  0   0   0  0
NE  0  0  79   0  0   0   0  0
NW  0  0   7   0  0   0   0  0
S   0  0   9   0  0   0   0  0
SE  0  0   4   0  0   0   0  0
SW  0  0  17   0  0   0   0  0
W   0  0  10   0  0   0   0  0
****************
Accuracy= 52.67%  Model Accuracy=32.17%
****************
Average precision = 6.58%  Model precision=21.03%
Average recall = 12.50%  Model recall=21.71%
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.526667     1.0  0.689956
NW      0.000000     0.0  0.000000
S       0.000000     0.0  0.000000
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Dummy**

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = DummyClassifier(strategy='most_frequent').fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)



    E  N  NE  NW  S  SE  SW  W
E   0  0  12   0  0   0   0  0
N   0  0  12   0  0   0   0  0
NE  0  0  79   0  0   0   0  0
NW  0  0   7   0  0   0   0  0
S   0  0   9   0  0   0   0  0
SE  0  0   4   0  0   0   0  0
SW  0  0  17   0  0   0   0  0
W   0  0  10   0  0   0   0  0
****************
Accuracy= 52.67%  Model Accuracy=32.17%
****************
Average precision = 6.58%  Model precision=21.03%
Average recall = 12.50%  Model recall=21.71%
       Precision  Recall        F1
W_DIR                             
E       0.000000     0.0  0.000000
N       0.000000     0.0  0.000000
NE      0.526667     1.0  0.689956
NW      0.000000     0.0  0.000000
S       0.000000     0.0  0.000000
SE      0.000000     0.0  0.000000
SW      0.000000     0.0  0.000000
W       0.000000     0.0  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Classifier Randomforest**

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)

    E  N  NE  NW  S  SE  SW  W
E   1  0   9   0  1   0   1  0
N   1  3   5   0  0   1   0  2
NE  1  3  70   0  1   2   2  0
NW  1  0   2   0  0   1   0  3
S   0  0   1   0  3   1   0  4
SE  0  0   3   0  0   0   1  0
SW  0  0   5   0  3   1   4  4
W   0  0   3   0  0   0   3  4
****************
Accuracy= 56.67%  Model Accuracy=32.17%
****************
Average precision = 30.48%  Model precision=21.03%
Average recall = 27.35%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.250000  0.083333  0.125000
N       0.500000  0.250000  0.333333
NE      0.714286  0.886076  0.790960
NW      0.000000  0.000000  0.000000
S       0.375000  0.333333  0.352941
SE      0.000000  0.000000  0.000000
SW      0.363636  0.235294  0.285714
W       0.235294  0.400000  0.296296


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Oversample minority class**

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data["label"], test_size=0.25, random_state=5)
X = pd.concat([x_train, y_train], axis=1)
not_NE = X[X["label"]!="NE"]
NE = X[X["label"]=="NE"]
not_NE_upsampled = resample(not_NE,
                          replace=True, # sample with replacement
                          n_samples=len(not_NE), # match number in majority class
                          random_state=27) # reproducible results
upsampled = pd.concat([NE, not_NE_upsampled])
y_train=upsampled["label"]
x_train=upsampled.drop("label",axis=1)
y_pred = RandomForestClassifier(n_estimators=20).fit(x_train, y_train).predict(x_test)
evaluate(y_test,y_pred)


**Generate synthetic samples**

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
print("Original train samples=",x_train.shape[0])
#y_pred = RandomForestClassifier(n_estimators=15).fit(x_train, y_train).predict(x_test)
#evaluate(y_test["label"],y_pred)

sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
print("Synthetic train samples=",x_train.shape[0])
print("****Randomforest****")
y_pred = RandomForestClassifier(n_estimators=15).fit(x_train, y_train).predict(x_test)
evaluate(y_test["label"],y_pred)
print("****SVM****")
y_pred=svm.SVC().fit(x_train,y_train).predict(x_test)
evaluate(y_test["label"],y_pred)


Original train samples= 450
Synthetic train samples= 2024
****Randomforest****
    E  N  NE  NW  S  SE  SW  W
E   0  1   6   1  1   1   0  0
N   1  4   3   0  0   0   0  2
NE  6  5  59   1  0   3   5  2
NW  1  0   1   0  0   0   3  2
S   0  0   0   1  3   1   5  2
SE  2  1   0   0  1   0   1  1
SW  1  1   1   0  4   0   5  0
W   1  0   2   2  0   0   5  2
****************
Accuracy= 48.67%  Model Accuracy=32.17%
****************
Average precision = 23.45%  Model precision=21.03%
Average recall = 24.52%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.333333  0.400000  0.363636
NE      0.819444  0.728395  0.771242
NW      0.000000  0.000000  0.000000
S       0.333333  0.250000  0.285714
SE      0.000000  0.000000  0.000000
SW      0.208333  0.416667  0.277778
W       0.181818  0.166667  0.173913
****SVM****


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


    E  N  NE  NW  S  SE  SW  W
E   0  0  10   0  0   0   0  0
N   0  0  10   0  0   0   0  0
NE  0  2  79   0  0   0   0  0
NW  0  0   7   0  0   0   0  0
S   0  0  10   0  1   0   1  0
SE  0  0   6   0  0   0   0  0
SW  0  0  10   0  0   0   2  0
W   0  0  12   0  0   0   0  0
****************
Accuracy= 54.67%  Model Accuracy=32.17%
****************
Average precision = 27.69%  Model precision=21.03%
Average recall = 15.32%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.000000  0.000000  0.000000
N       0.000000  0.000000  0.000000
NE      0.548611  0.975309  0.702222
NW      0.000000  0.000000  0.000000
S       1.000000  0.083333  0.153846
SE      0.000000  0.000000  0.000000
SW      0.666667  0.166667  0.266667
W       0.000000  0.000000  0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


**Cross validation K_folds**

In [0]:
clf=RandomForestClassifier(n_estimators=15)
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy"]
scores = cross_validate(clf, x_data, y_data["label"], scoring=scoring,
                        cv=5, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1: {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))

**Tuning SVM**

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
tuned_parameters = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
clf = GridSearchCV(svm.SVC(), tuned_parameters,scoring="f1_macro", cv=5).fit(x_train, y_train)
                    
print("Best parameters=",clf.best_params_)
print("Best F1=",clf.best_score_)
#SVC(C=1000, gamma=0.001, kernel='rbf')


Best parameters= {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
Best F1= 0.8573368990941034


**Evaluate SVM with best parameters**

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
y_pred=svm.SVC(C=1000, gamma=0.0001, kernel='rbf').fit(x_train,y_train).predict(x_test)
evaluate(y_test["label"],y_pred)

    E  N  NE  NW  S  SE  SW  W
E   4  1   5   0  0   0   0  0
N   2  2   4   1  0   0   0  1
NE  9  2  56   2  2   2   5  3
NW  1  1   1   0  0   1   1  2
S   1  1   1   1  3   2   3  0
SE  2  2   1   0  1   0   0  0
SW  0  1   1   0  3   0   7  0
W   1  0   2   2  1   0   3  3
****************
Accuracy= 50.00%  Model Accuracy=32.17%
****************
Average precision = 27.38%  Model precision=21.03%
Average recall = 29.68%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.200000  0.400000  0.266667
N       0.200000  0.200000  0.200000
NE      0.788732  0.691358  0.736842
NW      0.000000  0.000000  0.000000
S       0.300000  0.250000  0.272727
SE      0.000000  0.000000  0.000000
SW      0.368421  0.583333  0.451613
W       0.333333  0.250000  0.285714


**Tuning RandomForestClassifier**

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=27)
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
# Synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
rf_random.fit(x_train, y_train)
print("best parameters",rf_random.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.8min finished


best parameters {'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}


**Running Randomforest tuned**

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=27)
#Generate synthetic samples
sm = SMOTE(random_state=27,)
x_train, y_train = sm.fit_sample(x_train, y_train["label"])
clas=RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=70, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
y_pred = clas.fit(x_train, y_train).predict(x_test)
evaluate(y_test["label"],y_pred)

    E  N  NE  NW  S  SE  SW  W
E   2  0   7   1  0   0   1  1
N   0  3   8   0  0   1   0  2
NE  8  4  68   3  2   5   5  2
NW  0  0   1   2  0   1   2  2
S   1  1   0   1  4   1   3  1
SE  2  1   0   0  2   0   2  0
SW  0  1   1   1  3   0   7  2
W   1  0   3   2  1   1   5  2
****************
Accuracy= 48.89%  Model Accuracy=32.17%
****************
Average precision = 27.44%  Model precision=21.03%
Average recall = 28.32%  Model recall=21.71%
       Precision    Recall        F1
W_DIR                               
E       0.142857  0.166667  0.153846
N       0.300000  0.214286  0.250000
NE      0.772727  0.701031  0.735135
NW      0.200000  0.250000  0.222222
S       0.333333  0.333333  0.333333
SE      0.000000  0.000000  0.000000
SW      0.280000  0.466667  0.350000
W       0.166667  0.133333  0.148148


**Cross validation K_folds**

In [31]:
scoring = ['precision_macro', 'recall_macro','f1_macro',"accuracy"]
scores = cross_validate(clas, x_data, y_data["label"], scoring=scoring,
                        cv=5, return_train_score=False)
print("Accuracy: {:.2%} (+/- {:.2%})" .format (scores["test_accuracy"].mean(), scores["test_accuracy"].std() * 2))
print("Recall: {:.2%} (+/- {:.2%})" .format (scores["test_recall_macro"].mean(), scores["test_recall_macro"].std() * 2))
print("Precision: {:.2%} (+/- {:.2%})" .format (scores["test_precision_macro"].mean(), scores["test_precision_macro"].std() * 2))
print("f1: {:.2%} (+/-{:.2%} )".format (scores["test_f1_macro"].mean(), scores["test_f1_macro"].std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

Accuracy: 55.04% (+/- 9.52%)
Recall: 22.44% (+/- 4.29%)
Precision: 23.80% (+/- 6.88%)
f1: 21.89% (+/-4.63% )


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
